### Desafio 2

Geolocalizar eventos en noticias

Utilizando técnicas de tratamiento automático del lenguaje, el objetivo del desafío es estructurar la información de una noticia de la manera siguiente:

- event: principal evento descrito en la noticia
- category: categoria temática del evento
- address: dirección dónde occurió el evento en formato: calle número, comuna, país
- latitud: latitud del evento
- longitud: longitud del evento

Resultado: El formato del archivo CSV output que entregar contiene las columnas siguientes: id_news, event, category, address, latitud, longitud


#### Librerias


In [10]:
import pandas as pd
import spacy
from geopy.geocoders import Nominatim
from transformers import pipeline
import re

#### Preparacion


In [5]:
# Cargar el dataset de entrenamiento
df = pd.read_csv('./train_data.csv')

# Muestra aleatoria de 1000 registros
df_train = df.sample(n=1000, random_state=42)

# Barajar el DataFrame
df_train = df_train.sample(frac=1).reset_index(drop=True)

def format_df(df):
    """Convertir la columna de fechas al formato deseado."""
    df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y @ %H:%M:%S.%f')
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    return df

df_train = format_df(df_train)


#### Procesamiento


In [12]:
# Cargar el modelo de lenguaje natural en español
nlp = spacy.load('es_core_news_sm')

# pipeline de pregunta-respuesta
qa_model = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [13]:
def extract_info(row):
    """Extraer información relevante de la noticia."""
    # Procesar el texto con spaCy
    text = row['text']
    doc = nlp(text)
    
    # Definir la pregunta para el modelo de QA
    question_event = "¿De qué tipo de evento habla la noticia?"

    question_place = "¿Dónde ocurrió el evento?"

    
    
    # Obtener la respuesta utilizando el modelo de QA
    qa_response = qa_model(question=question_event, context=text)
    event = qa_response['answer']  # Extraer solo el texto de la respuesta
    
    # Inicializar variables para la categoría
    category = "Categoría simulada"  # Simular extracción de categoría

    # Inicializar una lista para las direcciones encontradas
    addresses = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
    
    # Formatear la dirección
    address = ", ".join(addresses) if addresses else "no encontrada"
    
    # Mostrar la noticia procesada y sus elementos
    print("--------")
    print(text)
    print(f"Noticia ID: {row['id_news']}")
    print(f"Evento: {event}")
    print(f"Categoría: {category}")
    print(f"Dirección: {address}")
    print("--------")
    
    return event, category, address

# Aplicar la función a cada fila del DataFrame
df_train[['event', 'category', 'address']] = df_train.apply(extract_info, axis=1, result_type='expand')

# Geocodificar las direcciones para obtener latitud y longitud
geolocator = Nominatim(user_agent="geoapiExercises")

def geocode_address(address):
    """Geocodificar una dirección para obtener latitud y longitud."""
    if address and address != "no encontrada":
        try:
            location = geolocator.geocode(address)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Error geocodificando {address}: {e}")
            return None, None
    return None, None  # Retornar None si no hay dirección válida

# Aplicar la geocodificación
df_train[['latitud', 'longitud']] = df_train['address'].apply(lambda x: pd.Series(geocode_address(x)))

# Seleccionar las columnas finales para el CSV de salida
output_df = df_train[['id_news', 'event', 'category', 'address', 'latitud', 'longitud']]

# Guardar el DataFrame de salida en un archivo CSV
output_df.to_csv('output.csv', index=False)

--------
(CNN) – El Comité Ejecutivo de la Unión de Federaciones Europeas de Fútbol (UEFA) realizó una reunión extraordinaria este viernes “tras la grave escalada de la situación de seguridad en Europa” y anunció que París sustituiría a la ciudad rusa de San Petersburgo como sede de la final de la Champions League. El encuentro se jugará el 28 de mayo en el Stade de France, el estadio nacional de Francia, ubicado justo al norte de París en Saint-Denis. Lee también: Avanza el ataque ruso: Se reportan fuertes explosiones en Kiev, capital de Ucrania En la reunión, “el Comité Ejecutivo de la UEFA también decidió que los clubes y equipos nacionales rusos y ucranianos que compitan en competiciones de la UEFA deberán jugar sus partidos como local en sedes neutrales hasta nuevo aviso”. The 2021/22 UEFA Men’s Champions League final will move from Saint Petersburg to Stade de France in Saint-Denis. The game will be played as initially scheduled on Saturday 28 May at 21:00 CET. Full statement: ⬇️

KeyboardInterrupt: 